# Definição de funções e imports necessários

In [1]:
import pandas as pd
import numpy as np
import json
import pymongo

### Funções

In [2]:
def readXLSX (path):
    return pd.read_excel(path, index_col=None);

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORD>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def deletingColumns (dataframe):
    columns = ["Região", 
               "UF", 
               "Código do Município", 
               "Nome do Município", 
               "Nome da Escola", 
               "Localização", 
               "Dependência Administrativa"];
    
    for column in columns:
        try:
            dataframe.pop(column)
        except:
            continue
    
def JSONstructure (data):    
    dataJSON = {"Taxa de distorção de idade-série por turma": 
                {
                "Última atualização": data["Ano"],
                "Ensino Fundamental 8 e 9 anos":
                    {
                        "Total": data["TotalF"],
                        "Anos Iniciais": data["Anos Iniciais"],
                        "Anos Finais": data["Anos Finais"],
                        "1 Ano": data["1° ano"],
                        "2 Ano": data["2° ano"],
                        "3 Ano": data["3° ano"],
                        "4 Ano": data["4° ano"],
                        "5 Ano": data["5° ano"],
                        "6 Ano": data["6° ano"],
                        "7 Ano": data["7° ano"],
                        "8 Ano": data["8° ano"],
                        "9 Ano": data["9° ano"]
                    },
                "Ensino Médio": 
                    {
                        "Total": data["TotalM"],
                        "1 série": data["1ª série"],
                        "2 série": data["2ª série"],
                        "3 série": data["3ª série"],
                        "4 série": data["4ª série"]
                    }
            }};

    return dataJSON;

def filterPelotas (dataframe):
    filter = dataframe["Nome do Município"] == "Pelotas"
    
    filterPelotas = dataframe[filter]
    filterPelotas.reset_index(drop=True, inplace=True)
    
    return filterPelotas

def insertIntoDB (dataframe, collection):
    for index, row in dataframe.iterrows():
        teste1 = JSONstructure(row);

        collection.update_one({"Código INEP": row["Código da Escola"]}, {"$set": teste1});

### Conexão com o BD:

In [3]:
collection = openDataBaseMongoDB()

### Extração dos dados

In [4]:
distortionRate = readXLSX(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\TaxaDistorcaoIdadeSerie.xlsx')

In [5]:
distortionRate.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,14.3,...,0,--,--,--,--,--,--,--,--,--
1,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,EMMEF 7 DE SETEMBRO,Rural,Municipal,5,...,0,--,--,--,--,--,--,--,--,--
2,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,EMEIEF ANA NERY,Urbana,Municipal,13.7,...,18.2,0,28.6,46.7,25,--,--,--,--,--
3,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Rural,Municipal,13.8,...,9.1,21.2,20,19.2,38.9,--,--,--,--,--
4,2019,Norte,RO,1100015,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Urbana,Estadual,15.6,...,14.1,19,22.3,14.3,--,--,--,--,--,--


In [6]:
distortionRate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132428 entries, 0 to 132427
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Ano                         132428 non-null  int64 
 1   Região                      132428 non-null  object
 2   UF                          132428 non-null  object
 3   Código do Município         132428 non-null  int64 
 4   Nome do Município           132428 non-null  object
 5   Código da Escola            132428 non-null  int64 
 6   Nome da Escola              132428 non-null  object
 7   Localização                 132428 non-null  object
 8   Dependência Administrativa  132428 non-null  object
 9   TotalF                      132428 non-null  object
 10  Anos Iniciais               132428 non-null  object
 11  Anos Finais                 132428 non-null  object
 12  1° ano                      132428 non-null  object
 13  2° ano                      1

### Transformação dos dados

* Filtro das escolas de Pelotas

In [7]:
distortionRate_Pelotas = filterPelotas(distortionRate)

In [8]:
distortionRate_Pelotas.head()

,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,TotalF,...,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,Sul,RS,4314407,Pelotas,43000304,COLEGIO TIRADENTES PELOTAS,Urbana,Estadual,--,...,--,--,--,--,--,13.1,6.7,23.1,8.3,--
1,2019,Sul,RS,4314407,Pelotas,43003222,EEF TRES VENDAS,Urbana,Privada,1.3,...,1.3,2.7,1.6,4.1,--,--,--,--,--,--
2,2019,Sul,RS,4314407,Pelotas,43012396,ESC EST ENS MED COLONIA DE PESCADORES Z 3,Rural,Estadual,--,...,--,--,--,--,--,79.5,93.1,50,55.6,--
3,2019,Sul,RS,4314407,Pelotas,43065376,EMEF WALDEMAR DENZER,Rural,Municipal,18,...,14.3,31.3,38.9,25,18.8,--,--,--,--,--
4,2019,Sul,RS,4314407,Pelotas,43100007,COLEGIO GONZAGA,Urbana,Privada,2,...,0.9,0,1.3,5.6,4.4,6.5,10,7.4,1.7,--


* Colunas desnecessárias

In [9]:
deletingColumns(distortionRate_Pelotas)

In [13]:
distortionRate_Pelotas.head()

,Ano,Código da Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,3° ano,4° ano,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,43000304,--,--,--,--,--,--,--,--,--,--,--,--,13.1,6.7,23.1,8.3,--
1,2019,43003222,1.3,0.5,2.9,0,0,1.1,0,1.3,2.7,1.6,4.1,--,--,--,--,--,--
2,2019,43012396,--,--,--,--,--,--,--,--,--,--,--,--,79.5,93.1,50,55.6,--
3,2019,43065376,18,3.1,29.1,0,0,0,0,14.3,31.3,38.9,25,18.8,--,--,--,--,--
4,2019,43100007,2,1.2,3,0,3.5,0.8,1.1,0.9,0,1.3,5.6,4.4,6.5,10,7.4,1.7,--


* Atribuir a valores não informado o valor nan

In [14]:
distortionRate_Pelotas = distortionRate_Pelotas.replace(regex=['--'], value = np.nan)

In [15]:
distortionRate_Pelotas.head()

,Ano,Código da Escola,TotalF,Anos Iniciais,Anos Finais,1° ano,2° ano,3° ano,4° ano,5° ano,6° ano,7° ano,8° ano,9° ano,TotalM,1ª série,2ª série,3ª série,4ª série
0,2019,43000304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.1,6.7,23.1,8.3,NaN
1,2019,43003222,1.3,0.5,2.9,0.0,0.0,1.1,0.0,1.3,2.7,1.6,4.1,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,43012396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.5,93.1,50.0,55.6,NaN
3,2019,43065376,18.0,3.1,29.1,0.0,0.0,0.0,0.0,14.3,31.3,38.9,25.0,18.8,NaN,NaN,NaN,NaN,NaN
4,2019,43100007,2.0,1.2,3.0,0.0,3.5,0.8,1.1,0.9,0.0,1.3,5.6,4.4,6.5,10.0,7.4,1.7,NaN


### Teste JSON

In [16]:
rowData = distortionRate_Pelotas.iloc[ 0 , : ]

In [17]:
JSONstructure(rowData)

{'Taxa de distorção de idade-série por turma': {'Última atualização': 2019.0,
  'Ensino Fundamental 8 e 9 anos': {'Total': nan,
   'Anos Iniciais': nan,
   'Anos Finais': nan,
   '1 Ano': nan,
   '2 Ano': nan,
   '3 Ano': nan,
   '4 Ano': nan,
   '5 Ano': nan,
   '6 Ano': nan,
   '7 Ano': nan,
   '8 Ano': nan,
   '9 Ano': nan},
  'Ensino Médio': {'Total': 13.1,
   '1 série': 6.7,
   '2 série': 23.1,
   '3 série': 8.3,
   '4 série': nan}}}

### Carregamento dos dados

* Inserindo no Banco de Dados (MongoDB)

In [18]:
insertIntoDB(distortionRate_Pelotas, collection)

In [19]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2620d181e7e7953bdb84af'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308,
 'Média de alunos por turma': {'Última atualização': 2019.0,
  'Educação Infantil': {'Total': nan, 'Creche': nan